### SelectFromModel
使用模型对特征进行选择,具体是利用类的feature_importances_或coef_属性

**参数:**
 - estimator:object;用来构建变压器的基本估算器。这既可以是拟合的（如果prefit设置为True），也可以是不拟合的估计量。拟合后，估算器必须具有 feature_importances_或coef_属性。
 - threshold:string, float,默认为None;用于特征选择的阈值。保留重要性更高或相等的要素，而其他要素则被丢弃。如果是浮点数,则制定阈值的绝对大小;如果为字符串,则可选mean(或者median),特征重要程度的均值(中值),如果为\"1.5\*mean\"则表示选择为均值的1.5倍;如果为None,则需要分情况讨论,如果该学习器存在一个penalty参数,且设置为l1,则默认阈值为1e-5,否则默认为mean.
 - prefit:bool,默认为为False;预设模型是否应该直接传递给构造函数。 如果为True，则必须直接调用transform，并且SelectFromModel不能与cross_val_score，GridSearchCV和用于克隆估算器的类似实用程序一起使用。 否则，使用拟合训练模型，然后进行变换以进行特征选择。
 - norm_orde:非零int，inf，-inf，默认值为1;在估算器的coef_属性的尺寸为2的情况下，用于过滤低于阈值的系数矢量的范数的顺序。
 - max_features:int或者None,默认为None;要选择的最大功能数。 要仅基于max_features选择，请设置threshold = -np.inf。

In [1]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
X = [[ 0.87, -1.34,  0.31 ],
     [-2.79, -0.02, -0.85 ],
     [-1.34, -0.48, -2.55 ],
     [ 1.92,  1.48,  0.65 ]]
y = [0, 1, 0, 1]
selector = SelectFromModel(estimator=LogisticRegression()).fit(X, y)
selector.estimator_.coef_

array([[-0.3252302 ,  0.83462377,  0.49750423]])

In [2]:
selector.threshold_

0.5524527319086916

In [3]:
selector.get_support()

array([False,  True, False])

In [4]:
selector.transform(X)

array([[-1.34],
       [-0.02],
       [-0.48],
       [ 1.48]])

### RFE
递归特征消除;
给定一个将权重分配给特征（例如线性模型的系数）的机器学习模型，递归特征消除（RFE）的目标是通过递归考虑越来越少的特征集来选择特征。首先，对估计器进行初始特征集训练，并通过coef_属性或属性获得每个特征的重要性 feature_importances_。然后，从当前功能集中删除最不重要的功能。在修剪的集上递归地重复该过程，直到最终达到所需的要选择的特征数量。

**参数:**
 - estimator:object;机器学习模型，必须拥有fit方法和coef_，feature_importances_两个属性
 - n_features_to_select:int或者None（默认为None）;选择保留的特征数量,如果为None,则保留一半
 - step:int 或者 float，默认为1;如果大于或等于1，则step对应于每次迭代要删除的（整数）个特征。如果在（0.0，1.0）之内，则step对应于每次迭代要删除的要素的百分比（向下舍入）。
 - verbose:int，默认为0;控制输出的详细程度。

In [1]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X, y)
selector.support_

array([ True,  True,  True,  True,  True, False, False, False, False,
       False])

In [2]:
selector.ranking_

array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])

### RFECV
通过消除递归特征和交叉验证最佳特征数选择来进行特征排名。

**参数：**
 - estimator:object;机器学习模型，必须拥有fit方法和coef_，feature_importances_两个属性
 - step:int或者float,默认为1;如果大于或等于1，则step对应于每次迭代要删除的（整数）个特征。如果在（0.0，1.0）之内，则step对应于每次迭代要删除的要素的百分比（向下舍入）。请注意，最后一次迭代可能删除的要素少于step才能达到min_features_to_select。
 - n_features_to_select:int,默认为1;最少要选择的特征。 即使无法逐步将原始特征计数与min_features_to_select之间的差异进行整除，也会始终对这一特征数进行评分。
 - cv:int或者一个交叉验证生成器或一个迭代器;None:使用默认的五折交叉验证;整数:用于指定折叠数;交叉验证分割器;可迭代的生成器(包含训练和测试);如果输入参数为整数或者None,并且y为二进制或者多分类,则使用sklearn.model_selection.StratifiedKFold,如果y既不是二进制也不是多类,则使用sklearn.model_selection.KFold。
 - scoring：string，可调用的函数或者None,默认为None;字符串（请参阅模型评估文档）或带签名的计分器可调用对象/函数 。scorer(estimator, X, y)。
 - verbose:int，默认为0;控制输出的详细程度。
 - n_jobs:int,None;使用的线程数，-1表示使用全部线程，但是可能会对可视化编程的监控线程造成影响。

In [3]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(X, y)
selector.support_

array([ True,  True,  True,  True,  True, False, False, False, False,
       False])

In [4]:
selector.ranking_

array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])